In [2]:
## imports
import pandas as pd
#import tensorflow as tf
from os import *
import numpy as np

In [3]:
import numpy as np

In [4]:
csv = pd.read_csv('../data/relevant/SPD_Crime_Data__2008-Present.csv')

df = pd.DataFrame()

wanted_cols = ['Offense Start DateTime', 'Offense Parent Group', 'Offense']

for col in wanted_cols:
    df[col] = csv[col]
    df = df.dropna()

df["Month"] = [pd.to_datetime(i).month for i in df['Offense Start DateTime']]

df["Day"] = [pd.to_datetime(i).day for i in df['Offense Start DateTime']]

df["Year"] = [pd.to_datetime(i).year for i in df['Offense Start DateTime']]

df.drop(columns = ['Offense Start DateTime'], inplace = True)

dfs = {}

In [5]:
def compress(frame: pd.DataFrame, wanted_columns: list, category: str) -> pd.DataFrame:
    df = pd.DataFrame()
    df[category] = frame[category]
    for col in wanted_columns:
        df[col] = frame[col]
    return df.value_counts().to_frame()

In [6]:
for i in ['Offense Parent Group', 'Offense']:
    dfs[i] = {}
    dfs[i]["ymd"] = compress(df, ["Day", "Month", "Year"], i)
    dfs[i]["ym"] = compress(df, ["Month", "Year"], i)
    dfs[i]["y"] = compress(df, ["Year"], i)

In [7]:
df

,Offense Parent Group,Offense,Month,Day,Year
0,DRUG/NARCOTIC OFFENSES,Drug/Narcotic Violations,2,5,2020
1,LARCENY-THEFT,Theft of Motor Vehicle Parts or Accessories,2,3,2020
2,ROBBERY,Robbery,2,2,2020
3,DESTRUCTION/DAMAGE/VANDALISM OF PROPERTY,Destruction/Damage/Vandalism of Property,2,5,2020
4,DRIVING UNDER THE INFLUENCE,Driving Under the Influence,2,5,2020
...,...,...,...,...,...
1033410,MOTOR VEHICLE THEFT,Motor Vehicle Theft,6,26,2013
1033411,LARCENY-THEFT,Shoplifting,2,14,2012
1033412,BURGLARY/BREAKING&ENTERING,Burglary/Breaking & Entering,2,20,2011
1033413,LARCENY-THEFT,Shoplifting,9,19,2010


In [8]:
df["Year"].unique()

array([2020, 2019, 2018, 2017, 2016, 2014, 2012, 2006, 2011, 2015, 2010,
       2013, 2009, 2002, 2007, 2008, 2004, 2005, 1977, 1991, 2000, 1998,
       1995, 2003, 2001, 1999, 1990, 1993, 1994, 1997, 1987, 1996, 1964,
       1992, 1980, 1908, 2021, 1979, 1915, 1929, 1974, 1984, 1989, 1953,
       1976, 1920, 1988, 2022, 1983, 2023, 1957])

In [9]:
def one_param(category):
    l = df[[category, "Year"]]
    c2i = l.value_counts().to_frame()
    i2c = pd.DataFrame(l.value_counts().to_frame().index.to_list())
    tt = {}
    for i in i2c.iloc:
        if i[0] not in tt.keys():
            tt[i[0]] = {}
        else:
            if i[1] not in tt[i[0]]:
                tt[i[0]][i[1]] = 0
            tt[i[0]][i[1]] += int(c2i.loc[i[0]].loc[i[1]])
        
    return pd.DataFrame(tt).sort_index(ascending = False)

In [10]:
one_param("Offense")

,Theft From Motor Vehicle,Burglary/Breaking & Entering,Identity Theft,Destruction/Damage/Vandalism of Property,Motor Vehicle Theft,All Other Larceny,Theft of Motor Vehicle Parts or Accessories,Simple Assault,Shoplifting,Trespass of Real Property,...,"Human Trafficking, Commercial Sex Acts",Drunkenness,Justifiable Homicide,Incest,Negligent Manslaughter,Betting/Wagering,Bribery,"Human Trafficking, Involuntary Servitude",Operating/Promoting/Assisting Gambling,Gambling Equipment Violation
2023,834.0,886.0,76.0,754.0,862.0,577.0,572.0,554.0,339.0,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022,8927.0,8666.0,751.0,7236.0,NaN,NaN,NaN,5074.0,4323.0,2047.0,...,12.0,NaN,4.0,NaN,1.0,NaN,NaN,NaN,1.0,NaN
2021,9333.0,9823.0,823.0,NaN,5329.0,6239.0,5709.0,4683.0,3949.0,1825.0,...,8.0,5.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN
2020,9140.0,NaN,NaN,6658.0,4942.0,6102.0,3150.0,4755.0,3166.0,2624.0,...,8.0,4.0,2.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN
2019,10757.0,7643.0,660.0,5804.0,3991.0,6667.0,1405.0,5593.0,4523.0,3914.0,...,NaN,3.0,2.0,2.0,4.0,NaN,NaN,NaN,NaN,NaN
2018,11714.0,8105.0,104.0,6651.0,4099.0,6560.0,1458.0,NaN,NaN,NaN,...,NaN,NaN,2.0,5.0,NaN,1.0,1.0,NaN,1.0,NaN
2017,12516.0,7790.0,48.0,6644.0,3705.0,5935.0,1393.0,5630.0,4767.0,3909.0,...,2.0,NaN,3.0,2.0,NaN,3.0,2.0,NaN,NaN,NaN
2016,13191.0,7549.0,35.0,6322.0,4175.0,5569.0,1430.0,5475.0,4809.0,2832.0,...,NaN,NaN,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
2015,12683.0,7593.0,22.0,5983.0,3873.0,6144.0,1170.0,5278.0,4423.0,2189.0,...,NaN,NaN,2.0,1.0,NaN,2.0,1.0,NaN,NaN,NaN
2014,NaN,7135.0,16.0,5750.0,5497.0,6358.0,1529.0,5060.0,4304.0,1367.0,...,NaN,NaN,5.0,3.0,NaN,1.0,NaN,NaN,NaN,NaN


In [11]:
def two_param(category):
    l = df[[category, "Month", "Year"]]
    c2i = l.value_counts().to_frame()
    i2c = pd.DataFrame(l.value_counts().to_frame().index.to_list())
    tt = {}
    op = one_param(category)
    for i in i2c.iloc:
        if i[0] not in tt.keys():
            tt[i[0]] = {}
        else:
            if i[1] not in tt[i[0]]:
                tt[i[0]][i[1]] = {}
            else:
                if i[2] not in tt[i[0]][i[1]]:
                    tt[i[0]][i[1]][i[2]] = 0
                tt[i[0]][i[1]][i[2]] += int(c2i.loc[i[0]].loc[i[1]].loc[i[2]]) / op[i[0]][i[2]]
    return pd.DataFrame(tt).sort_index(ascending = False)

In [12]:
two_param("Offense")

,Identity Theft,Theft From Motor Vehicle,Impersonation,Burglary/Breaking & Entering,All Other Larceny,Destruction/Damage/Vandalism of Property,Motor Vehicle Theft,Theft of Motor Vehicle Parts or Accessories,Simple Assault,Shoplifting,...,"Human Trafficking, Commercial Sex Acts",Justifiable Homicide,Incest,Drunkenness,Betting/Wagering,Gambling Equipment Violation,"Human Trafficking, Involuntary Servitude",Bribery,Negligent Manslaughter,Operating/Promoting/Assisting Gambling
12,"{2019: 0.09848484848484848, 2022: 0.0812250332...","{2014: nan, 2015: 0.09027832531735394, 2013: 0...","{2014: nan, 2013: 0.09950248756218906, 2017: 0...","{2021: 0.09121449658963657, 2018: 0.0889574336...","{2014: 0.09059452658068574, 2017: 0.0960404380...","{2018: 0.08284468500977296, 2020: 0.0805046560...","{2021: 0.10020641771439294, 2020: 0.0924726831...","{2021: 0.07391837449640917, 2022: nan, 2008: 0...","{2018: nan, 2017: 0.08099467140319716, 2019: 0...","{2015: 0.09314944607732309, 2016: 0.0813058847...",...,{2019: nan},"{2017: 0.3333333333333333, 2015: 0.5}","{2016: 0.25, 2018: 0.2, 2021: 1.0}",NaN,{2017: 0.3333333333333333},NaN,NaN,{2009: 0.5},"{2019: 0.25, 2020: nan}",NaN
11,"{2019: 0.1, 2021: 0.06439854191980558, 2022: 0...","{2016: 0.09195663710105374, 2009: 0.0950003961...","{2015: 0.05104895104895105, 2017: 0.0677179962...","{2021: 0.08999287386745393, 2012: 0.1090409285...","{2018: 0.08658536585365853, 2019: 0.0824958752...","{2018: 0.08239362501879417, 2016: 0.0852578298...","{2021: 0.09401388628260461, 2020: 0.0973290165...","{2022: nan, 2020: 0.13365079365079366, 2008: 0...","{2008: 0.09100817438692098, 2017: 0.0847246891...","{2016: 0.08296943231441048, 2019: 0.0875525093...",...,NaN,{},{2016: 0.25},{2021: 0.2},{2020: 1.0},NaN,NaN,{},{},{}
10,"{2022: 0.09720372836218376, 2019: 0.1060606060...","{2009: 0.09935821250297124, 2014: nan, 2013: 0...","{2016: 0.09598031173092698, 2014: nan, 2015: 0...","{2017: 0.09409499358151477, 2022: 0.0838910685...","{2014: 0.0887071406102548, 2019: 0.08414579271...","{2017: 0.09391932570740517, 2008: 0.0827645051...","{2020: 0.11027923917442331, 2014: 0.0980534837...","{2022: nan, 2020: 0.1511111111111111, 2008: 0....","{2018: nan, 2017: 0.08632326820603908, 2019: 0...","{2016: 0.08359326263256395, 2018: nan, 2013: 0...",...,{2020: 0.125},{2016: 0.5},{},{},{},NaN,NaN,{2017: 0.5},{},NaN
9,"{2019: 0.09545454545454546, 2022: 0.0838881491...","{2015: 0.09043601671528818, 2014: nan, 2016: 0...","{2016: 0.07383100902378999, 2008: 0.0905077262...","{2021: 0.08052529777053853, 2017: 0.0844672657...","{2021: 0.09665010418336271, 2018: 0.0891768292...","{2008: 0.0947098976109215, 2020: 0.09282066686...","{2021: 0.10452242446988178, 2020: 0.1058276001...","{2022: nan, 2020: 0.12222222222222222, 2008: 0...","{2018: nan, 2019: 0.08653674235651708, 2016: 0...","{2017: 0.07530941892175372, 2014: 0.0827137546...",...,"{2020: 0.125, 2019: nan}",NaN,{},{},{2021: 1.0},NaN,NaN,{},"{2020: nan, 2021: 1.0}",{}
8,"{2019: 0.11060606060606061, 2021: 0.0850546780...","{2014: nan, 2017: 0.08692873122403323, 2015: 0...","{2015: 0.05804195804195804, 2014: nan, 2008: 0...","{2021: 0.07421358037259493, 2018: 0.0856261566...","{2019: 0.09569521523923803, 2014: 0.0967285309...","{2020: 0.10198257735055573, 2008: 0.0937144482...","{2020: 0.10056657223796034, 2021: 0.0917620566...","{2022: nan, 2020: 0.10634920634920635, 2010: 0...","{2018: nan, 2016: 0.09351598173515982, 2019: 0...","{2017: 0.08726662471155863, 2012: 0.0937996820...",...,"{2021: 0.125, 2022: 0.08333333333333333}",{2014: 0.2},{},NaN,{},NaN,NaN,NaN,{},NaN
7,"{2019: 0.11969696969696969, 2021: 0.0595382746...","{2014: nan, 2018: 0.0862216151613454, 2016: 0....","{2015: 0.06293706293706294, 2016: 0.0664479081...","{2021: 0.07798025043265805, 2022: 0.0864297253...","{2018: 0.1, 2019: 0.0967451627418629, 2021: 0....","{2022: 0.09480375898286346, 2021: nan, 2020: 0...","{2014: 0.09787156630889576, 2020: 0.0896398219...","{2022: nan, 2020: 0.07238095238095238, 2011: 0...","{2017

In [ ]:
l = df[["Offense", "Month", "Year"]]

In [247]:
c2i = l.value_counts().to_frame()
i2c = pd.DataFrame(l.value_counts().to_frame().index.to_list())

In [256]:
i2c

,0,1,2
0,Identity Theft,5,2020
1,Identity Theft,3,2020
2,Theft From Motor Vehicle,1,2010
3,Theft From Motor Vehicle,4,2014
4,Theft From Motor Vehicle,11,2015
...,...,...,...
8718,Peeping Tom,7,2022
8719,Peeping Tom,7,2019
8720,Peeping Tom,7,2018
8721,Peeping Tom,7,2016


In [ ]:
tt = {}
for i in i2c.iloc:
    if i[0] not in tt.keys():
        tt[i[0]] = {}
    else:
        if i[1] not in tt[i[0]]:
            tt[i[0]][i[1]] = {}
        else:
            if i[2] not in tt[i[0]][i[1]]:
                tt[i[0]][i[1]][i[2]] = 0
            tt[i[0]][i[1]][i[2]] += int(c2i.loc[i[0]].loc[i[1]].loc[i[2]]) / one_param("Offense")[i[0]][i[2]]

In [255]:
c2i

0
Offense                  Month Year      
Identity Theft           5     2020  6335
                         3     2020  1420
Theft From Motor Vehicle 1     2010  1393
                         4     2014  1381
                         11    2015  1321
...                                   ...
Peeping Tom              7     2022     1
                               2019     1
                               2018     1
                               2016     1
Intimidation             11    2007     1

[8723 rows x 1 columns]

In [254]:
pd.DataFrame(l.value_counts().to_frame().index.to_list()).sort_values(1)

,0,1,2
7156,Credit Card/Automated Teller Machine Fraud,1,2001
3616,False Pretenses/Swindle/Confidence Game,1,2023
1511,Trespass of Real Property,1,2022
8274,Sexual Assault With An Object,1,2008
1506,Aggravated Assault,1,2021
...,...,...,...
3408,Stolen Property Offenses,12,2017
3403,Identity Theft,12,2021
3402,"Family Offenses, Nonviolent",12,2012
3356,False Pretenses/Swindle/Confidence Game,12,2008


In [244]:
one_param('Offense Parent Group')

,LARCENY-THEFT,FRAUD OFFENSES,ASSAULT OFFENSES,BURGLARY/BREAKING&ENTERING,DESTRUCTION/DAMAGE/VANDALISM OF PROPERTY,MOTOR VEHICLE THEFT,TRESPASS OF REAL PROPERTY,DRUG/NARCOTIC OFFENSES,ROBBERY,DRIVING UNDER THE INFLUENCE,...,EXTORTION/BLACKMAIL,HOMICIDE OFFENSES,PORNOGRAPHY/OBSCENE MATERIAL,ANIMAL CRUELTY,PEEPING TOM,"SEX OFFENSES, CONSENSUAL",HUMAN TRAFFICKING,DRUNKENNESS,GAMBLING OFFENSES,BRIBERY
2023,2508.0,252.0,1206.0,886.0,754.0,862.0,200.0,87.0,180.0,176.0,...,10.0,7.0,4.0,3.0,2.0,NaN,NaN,NaN,NaN,NaN
2022,NaN,2549.0,NaN,8666.0,7236.0,NaN,2047.0,671.0,1758.0,1301.0,...,NaN,NaN,45.0,37.0,8.0,7.0,12.0,NaN,1.0,NaN
2021,27172.0,2650.0,11022.0,9823.0,NaN,5329.0,1825.0,393.0,1753.0,990.0,...,112.0,43.0,34.0,NaN,5.0,8.0,8.0,5.0,1.0,1.0
2020,23322.0,NaN,10296.0,NaN,6658.0,4942.0,2624.0,1003.0,1489.0,998.0,...,105.0,55.0,39.0,26.0,15.0,5.0,10.0,4.0,1.0,1.0
2019,26035.0,3463.0,11183.0,7643.0,5804.0,3991.0,3914.0,1637.0,1555.0,1408.0,...,95.0,38.0,50.0,17.0,15.0,7.0,NaN,3.0,NaN,NaN
2018,27035.0,3722.0,11330.0,8105.0,6651.0,4099.0,NaN,2165.0,1701.0,1390.0,...,102.0,34.0,NaN,NaN,16.0,10.0,NaN,NaN,2.0,1.0
2017,26977.0,3891.0,10311.0,7790.0,6644.0,3705.0,3909.0,2325.0,1541.0,1381.0,...,79.0,30.0,22.0,NaN,NaN,14.0,2.0,NaN,3.0,2.0
2016,27170.0,3674.0,9794.0,7549.0,6322.0,4175.0,2832.0,1849.0,1553.0,1166.0,...,68.0,22.0,13.0,NaN,9.0,15.0,NaN,NaN,NaN,NaN
2015,26632.0,3849.0,9393.0,7593.0,5983.0,3873.0,2189.0,1612.0,1548.0,1025.0,...,40.0,28.0,14.0,NaN,14.0,4.0,NaN,NaN,2.0,1.0
2014,28324.0,4919.0,9010.0,7135.0,5750.0,5497.0,1367.0,1230.0,1579.0,1014.0,...,20.0,28.0,13.0,NaN,9.0,10.0,NaN,NaN,1.0,NaN


In [240]:
one_param("Offense")

,Theft From Motor Vehicle,Burglary/Breaking & Entering,Identity Theft,Destruction/Damage/Vandalism of Property,Motor Vehicle Theft,All Other Larceny,Theft of Motor Vehicle Parts or Accessories,Simple Assault,Shoplifting,Trespass of Real Property,...,"Human Trafficking, Commercial Sex Acts",Drunkenness,Justifiable Homicide,Incest,Negligent Manslaughter,Betting/Wagering,Bribery,"Human Trafficking, Involuntary Servitude",Operating/Promoting/Assisting Gambling,Gambling Equipment Violation
2023,834.0,886.0,76.0,754.0,862.0,577.0,572.0,554.0,339.0,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022,8927.0,8666.0,751.0,7236.0,NaN,NaN,NaN,5074.0,4323.0,2047.0,...,12.0,NaN,4.0,NaN,1.0,NaN,NaN,NaN,1.0,NaN
2021,9333.0,9823.0,823.0,NaN,5329.0,6239.0,5709.0,4683.0,3949.0,1825.0,...,8.0,5.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN
2020,9140.0,NaN,NaN,6658.0,4942.0,6102.0,3150.0,4755.0,3166.0,2624.0,...,8.0,4.0,2.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN
2019,10757.0,7643.0,660.0,5804.0,3991.0,6667.0,1405.0,5593.0,4523.0,3914.0,...,NaN,3.0,2.0,2.0,4.0,NaN,NaN,NaN,NaN,NaN
2018,11714.0,8105.0,104.0,6651.0,4099.0,6560.0,1458.0,NaN,NaN,NaN,...,NaN,NaN,2.0,5.0,NaN,1.0,1.0,NaN,1.0,NaN
2017,12516.0,7790.0,48.0,6644.0,3705.0,5935.0,1393.0,5630.0,4767.0,3909.0,...,2.0,NaN,3.0,2.0,NaN,3.0,2.0,NaN,NaN,NaN
2016,13191.0,7549.0,35.0,6322.0,4175.0,5569.0,1430.0,5475.0,4809.0,2832.0,...,NaN,NaN,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
2015,12683.0,7593.0,22.0,5983.0,3873.0,6144.0,1170.0,5278.0,4423.0,2189.0,...,NaN,NaN,2.0,1.0,NaN,2.0,1.0,NaN,NaN,NaN
2014,NaN,7135.0,16.0,5750.0,5497.0,6358.0,1529.0,5060.0,4304.0,1367.0,...,NaN,NaN,5.0,3.0,NaN,1.0,NaN,NaN,NaN,NaN


In [185]:
l = df[["Offense", "Year"]]

In [ ]:
l2 = df[]

In [207]:
c2i = l.value_counts().to_frame()

In [208]:
i2c = pd.DataFrame(l.value_counts().to_frame().index.to_list())

In [222]:
tt = {}

In [224]:
c2i

0
Offense                   Year       
Theft From Motor Vehicle  2014  13887
                          2016  13191
                          2015  12683
                          2009  12621
                          2017  12516
...                               ...
                          2002      1
                          2003      1
Hacking/Computer Invasion 2017      1
                          2018      1
Aggravated Assault        1997      1

[1078 rows x 1 columns]

In [235]:
for i in i2c.iloc:
    if i[0] not in tt.keys():
        tt[i[0]] = {}
    else:
        if i[1] not in tt[i[0]]:
            tt[i[0]][i[1]] = 0
        tt[i[0]][i[1]] += int(c2i.loc[i[0]].loc[i[1]])

In [234]:
pd.DataFrame(tt).sort_index(ascending = False)

,Theft From Motor Vehicle,Burglary/Breaking & Entering,Identity Theft,Destruction/Damage/Vandalism of Property,Motor Vehicle Theft,All Other Larceny,Theft of Motor Vehicle Parts or Accessories,Simple Assault,Shoplifting,Trespass of Real Property,...,"Human Trafficking, Commercial Sex Acts",Drunkenness,Justifiable Homicide,Incest,Negligent Manslaughter,Betting/Wagering,Bribery,"Human Trafficking, Involuntary Servitude",Operating/Promoting/Assisting Gambling,Gambling Equipment Violation
2023,834.0,886.0,76.0,754.0,862.0,577.0,572.0,554.0,339.0,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022,8927.0,8666.0,751.0,7236.0,NaN,NaN,NaN,5074.0,4323.0,2047.0,...,12.0,NaN,4.0,NaN,1.0,NaN,NaN,NaN,1.0,NaN
2021,9333.0,9823.0,823.0,NaN,5329.0,6239.0,5709.0,4683.0,3949.0,1825.0,...,8.0,5.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN
2020,9140.0,NaN,NaN,6658.0,4942.0,6102.0,3150.0,4755.0,3166.0,2624.0,...,8.0,4.0,2.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN
2019,10757.0,7643.0,660.0,5804.0,3991.0,6667.0,1405.0,5593.0,4523.0,3914.0,...,NaN,3.0,2.0,2.0,4.0,NaN,NaN,NaN,NaN,NaN
2018,11714.0,8105.0,104.0,6651.0,4099.0,6560.0,1458.0,NaN,NaN,NaN,...,NaN,NaN,2.0,5.0,NaN,1.0,1.0,NaN,1.0,NaN
2017,12516.0,7790.0,48.0,6644.0,3705.0,5935.0,1393.0,5630.0,4767.0,3909.0,...,2.0,NaN,3.0,2.0,NaN,3.0,2.0,NaN,NaN,NaN
2016,13191.0,7549.0,35.0,6322.0,4175.0,5569.0,1430.0,5475.0,4809.0,2832.0,...,NaN,NaN,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
2015,12683.0,7593.0,22.0,5983.0,3873.0,6144.0,1170.0,5278.0,4423.0,2189.0,...,NaN,NaN,2.0,1.0,NaN,2.0,1.0,NaN,NaN,NaN
2014,NaN,7135.0,16.0,5750.0,5497.0,6358.0,1529.0,5060.0,4304.0,1367.0,...,NaN,NaN,5.0,3.0,NaN,1.0,NaN,NaN,NaN,NaN


In [ ]:
newdf = pd.DataFrame()
newdf["year"] = []
for i in range(len(l.Offense) - 1):
    if l.iloc[i]["Offense"] not in newdf.columns:
        newdf[l.iloc[i]["Offense"]] = []
    else:
        

In [75]:
dfs # MARIA SHOULD LOOK AT THIS

{'Offense Parent Group': {'ymd':                                        0
  Offense Parent Group Day Month Year     
  FRAUD OFFENSES       18  5     2020  593
                       8   3     2020  521
                       15  5     2020  507
                       14  5     2020  457
                       11  5     2020  411
  ...                                  ...
  EMBEZZLEMENT         2   7     2014    1
  SEX OFFENSES         4   1     2013    1
                                 2012    1
                                 2010    1
  ANIMAL CRUELTY       1   1     2023    1
  
  [87153 rows x 1 columns],
  'ym':                                         0
  Offense Parent Group     Month Year      
  FRAUD OFFENSES           5     2020  6519
  LARCENY-THEFT            10    2021  2753
                           3     2022  2653
                           1     2022  2624
                           4     2014  2592
  ...                                   ...
  SEX OFFENSES, CONSE

In [63]:
compress(df, ["Year", "Month", "Day"], "Offense")

0
Offense                  Year Month Day     
Identity Theft           2020 5     18   585
                              3     8    516
                              5     15   500
                                    14   453
                                    11   406
...                                      ...
Fondling                 2018 1     29     1
Stolen Property Offenses 2022 6     20     1
Fondling                 2018 1     23     1
Stolen Property Offenses 2022 6     22     1
Aggravated Assault       1997 3     12     1

[139112 rows x 1 columns]

In [59]:
df.value_counts().to_frame()

0
Offense Parent Group        Offense                     Month Day Year     
FRAUD OFFENSES              Identity Theft              5     18  2020  585
                                                        3     8   2020  516
                                                        5     15  2020  500
                                                              14  2020  453
                                                              11  2020  406
...                                                                     ...
FAMILY OFFENSES, NONVIOLENT Family Offenses, Nonviolent 7     8   2010    1
                                                              7   2018    1
                                                                  2016    1
                                                                  2015    1
ANIMAL CRUELTY              Animal Cruelty              1     1   2023    1

[139112 rows x 1 columns]

In [47]:
df

,Offense Start DateTime,Offense Parent Group,Offense
0,02/05/2020 10:10:00 AM,DRUG/NARCOTIC OFFENSES,Drug/Narcotic Violations
1,02/03/2020 08:00:00 AM,LARCENY-THEFT,Theft of Motor Vehicle Parts or Accessories
2,02/02/2020 08:30:00 PM,ROBBERY,Robbery
3,02/05/2020 01:17:00 AM,DESTRUCTION/DAMAGE/VANDALISM OF PROPERTY,Destruction/Damage/Vandalism of Property
4,02/05/2020 12:51:21 AM,DRIVING UNDER THE INFLUENCE,Driving Under the Influence
...,...,...,...
1033410,06/26/2013 11:00:00 AM,MOTOR VEHICLE THEFT,Motor Vehicle Theft
1033411,02/14/2012 03:04:00 PM,LARCENY-THEFT,Shoplifting
1033412,02/20/2011 08:00:00 PM,BURGLARY/BREAKING&ENTERING,Burglary/Breaking & Entering
1033413,09/19/2010 04:59:00 PM,LARCENY-THEFT,Shoplifting


In [42]:
csv.columns

Index(['Report Number', 'Offense ID', 'Offense Start DateTime',
       'Offense End DateTime', 'Report DateTime', 'Group A B',
       'Crime Against Category', 'Offense Parent Group', 'Offense',
       'Offense Code', 'Precinct', 'Sector', 'Beat', 'MCPP',
       '100 Block Address', 'Longitude', 'Latitude'],
      dtype='object')

In [39]:
df['Offense Start DateTime'].isna().value_counts()

False    1032212
Name: Offense Start DateTime, dtype: int64

In [27]:
## datasets, data cleanup, test-train split
ds = pd.read_csv('../data/relevant/SPD_Crime_Data__2008-Present.csv')

## cleaning up the dates to have month, day, year columns
## also splitting it up into the layers of data
## which includes year, month, day

ds = ds[ds['Offense Start DateTime'] != None]

# dateTime = ds['Offense Start DateTime'].dropna()

ds["Month"] = [pd.to_datetime(i).month for i in ds['Offense Start DateTime']]

ds["Day"] = [pd.to_datetime(i).day for i in ds['Offense Start DateTime']]

ds["Year"] = [pd.to_datetime(i).year for i in ds['Offense Start DateTime']]

ds.loc[:, 'Crime Against Category':'Year']

,Crime Against Category,Offense Parent Group,Offense,Offense Code,Precinct,Sector,Beat,MCPP,100 Block Address,Longitude,Latitude,Month,Day,Year
0,SOCIETY,DRUG/NARCOTIC OFFENSES,Drug/Narcotic Violations,35A,W,Q,Q1,MAGNOLIA,32XX BLOCK OF 23RD AVE W,-122.385974,47.649387,2.0,5.0,2020.0
1,PROPERTY,LARCENY-THEFT,Theft of Motor Vehicle Parts or Accessories,23G,N,J,J3,ROOSEVELT/RAVENNA,63XX BLOCK OF 5TH AVE NE,-122.323399,47.675118,2.0,3.0,2020.0
2,PROPERTY,ROBBERY,Robbery,120,N,U,U3,ROOSEVELT/RAVENNA,26TH AVE NE / NE BLAKELEY ST,-122.299552,47.666384,2.0,2.0,2020.0
3,PROPERTY,DESTRUCTION/DAMAGE/VANDALISM OF PROPERTY,Destruction/Damage/Vandalism of Property,290,W,Q,Q1,MAGNOLIA,22XX BLOCK OF W RAYE ST,-122.384865,47.642927,2.0,5.0,2020.0
4,SOCIETY,DRIVING UNDER THE INFLUENCE,Driving Under the Influence,90D,N,B,B2,BALLARD SOUTH,NW 46TH ST / 8TH AVE NW,-122.366195,47.662193,2.0,5.0,2020.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033410,PROPERTY,MOTOR VEHICLE THEFT,Motor Vehicle Theft,240,W,Q,Q3,QUEEN ANNE,1ST AVE N / VALLEY ST,-122.355386,47.626251,6.0,26.0,2013.0
1033411,PROPERTY,LARCENY-THEFT,Shoplifting,23C,N,U,U1,ROOSEVELT/RAVENNA,73XX BLOCK OF ROOSEVELT WAY NE,-122.317464,47.681894,2.0,14.0,2012.0
1033412,PROPERTY,BURGLARY/BREAKING&ENTERING,Burglary/Breaking & Entering,220,S,S,S3,RAINIER VIEW,100XX BLOCK OF M L KING JR WAY S,-122.278352,47.510453,2.0,20.0,2011.0
1033413,PROPERTY,LARCENY-THEFT,Shoplifting,23C,W,M,M3,DOWNTOWN COMMERCIAL,5XX BLOCK OF PIKE ST,-122.335018,47.610909,9.0,19.0,2010.0


In [ ]:
## train-test split
## to-do: for dsYear, Month, Day, create 3 splits
## each of 3 splits will be split 75% train and 25% test

In [ ]:
## polynomial regression
## to-do: learn how to do polynomial regression ,_,

In [ ]:
s = 'jacbja'

print(s[0:s.find('a')])
print(s[0:s.find('b')][0:s.find('a')])

j


In [5]:
ds = pd.read_csv('../data/relevant/SPD_Crime_Data__2008-Present.csv')

In [20]:
ds['Offense Start DateTime'].dropna()

0          02/05/2020 10:10:00 AM
1          02/03/2020 08:00:00 AM
2          02/02/2020 08:30:00 PM
3          02/05/2020 01:17:00 AM
4          02/05/2020 12:51:21 AM
                    ...          
1033410    06/26/2013 11:00:00 AM
1033411    02/14/2012 03:04:00 PM
1033412    02/20/2011 08:00:00 PM
1033413    09/19/2010 04:59:00 PM
1033414    02/25/2010 06:00:00 PM
Name: Offense Start DateTime, Length: 1032212, dtype: object

In [17]:
ds['Report DateTime'].isnull().value_counts()

False    1033415
Name: Report DateTime, dtype: int64

In [25]:
ds["Year"] = [pd.to_datetime(i).year for i in ds['Offense Start DateTime']]

In [26]:
ds

,Report Number,Offense ID,Offense Start DateTime,Offense End DateTime,Report DateTime,Group A B,Crime Against Category,Offense Parent Group,Offense,Offense Code,Precinct,Sector,Beat,MCPP,100 Block Address,Longitude,Latitude,Month,Day,Year
0,2020-044620,12605873663,02/05/2020 10:10:00 AM,NaN,02/05/2020 11:24:31 AM,A,SOCIETY,DRUG/NARCOTIC OFFENSES,Drug/Narcotic Violations,35A,W,Q,Q1,MAGNOLIA,32XX BLOCK OF 23RD AVE W,-122.385974,47.649387,2.0,5.0,2020.0
1,2020-044452,12605598696,02/03/2020 08:00:00 AM,02/04/2020 08:00:00 AM,02/05/2020 10:06:28 AM,A,PROPERTY,LARCENY-THEFT,Theft of Motor Vehicle Parts or Accessories,23G,N,J,J3,ROOSEVELT/RAVENNA,63XX BLOCK OF 5TH AVE NE,-122.323399,47.675118,2.0,3.0,2020.0
2,2020-044465,12605567653,02/02/2020 08:30:00 PM,02/02/2020 09:30:00 PM,02/05/2020 09:39:33 AM,A,PROPERTY,ROBBERY,Robbery,120,N,U,U3,ROOSEVELT/RAVENNA,26TH AVE NE / NE BLAKELEY ST,-122.299552,47.666384,2.0,2.0,2020.0
3,2020-044225,12605174036,02/05/2020 01:17:00 AM,02/05/2020 02:21:00 AM,02/05/2020 03:30:55 AM,A,PROPERTY,DESTRUCTION/DAMAGE/VANDALISM OF PROPERTY,Destruction/Damage/Vandalism of Property,290,W,Q,Q1,MAGNOLIA,22XX BLOCK OF W RAYE ST,-122.384865,47.642927,2.0,5.0,2020.0
4,2020-044076,12605081469,02/05/2020 12:51:21 AM,NaN,02/05/2020 12:51:31 AM,B,SOCIETY,DRIVING UNDER THE INFLUENCE,Driving Under the Influence,90D,N,B,B2,BALLARD SOUTH,NW 46TH ST / 8TH AVE NW,-122.366195,47.662193,2.0,5.0,2020.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033410,2013-227022,7682354808,06/26/2013 11:00:00 AM,06/29/2013 06:30:00 AM,06/29/2013 09:43:00 AM,A,PROPERTY,MOTOR VEHICLE THEFT,Motor Vehicle Theft,240,W,Q,Q3,QUEEN ANNE,1ST AVE N / VALLEY ST,-122.355386,47.626251,6.0,26.0,2013.0
1033411,2012-045494,7672915592,02/14/2012 03:04:00 PM,NaN,02/14/2012 03:04:00 PM,A,PROPERTY,LARCENY-THEFT,Shoplifting,23C,N,U,U1,ROOSEVELT/RAVENNA,73XX BLOCK OF ROOSEVELT WAY NE,-122.317464,47.681894,2.0,14.0,2012.0
1033412,2011-061224,41519143023,02/20/2011 08:00:00 PM,02/21/2011 01:55:00 PM,02/21/2011 01:55:00 PM,A,PROPERTY,BURGLARY/BREAKING&ENTERING,Burglary/Breaking & Entering,220,S,S,S3,RAINIER VIEW,100XX BLOCK OF M L KING JR WAY S,-122.278352,47.510453,2.0,20.0,2011.0
1033413,2010-328592,7692227482,09/19/2010 04:59:00 PM,NaN,09/19/2010 04:59:00 PM,A,PROPERTY,LARCENY-THEFT,Shoplifting,23C,W,M,M3,DOWNTOWN COMMERCIAL,5XX BLOCK OF PIKE ST,-122.335018,47.610909,9.0,19.0,2010.0


In [13]:
## polynomial regression

# split the data 50/50

# for i in range 10
#     create the model 
#     calculate mse for 1/2 and 2/2
#     if abs(mse1-mse2) < curmin curmin = abs(mse1-mse2) and bestmodel = curmodel 

# settle on the last model

#------------LOOK AT THIS CODE!!!------------#

# X1 = []  # for years 1/2
# y1 = []  # for occurences in that year 1/2

# X2 = []  # for years 2/2
# y2 = []  # for occurences in that year 2/2

# bestmodel = None
# curmin = int("inf")

# for i in range(1, 11):
#     curmodel = PolynomialRegression(degree = i)
#     curmodel.fit(X1, y1)
#     msetrain = mse(y1, curmodel.predict(X1))
#     msetest = mse(y2, curmodel.predict(X2))

#     diff = abs(msetrain-msetest)

#     if diff < curmin:
#         curmin = diff
#         bestmodel = curmodel  

#--------------------------------------------#


# BELOW IS HOW WE WOULD DO IT WITHOUT OPTIMIZING POLYNOMIAL DEGREE

# from sklearn.linear_model import PolynomialRegression

# X = []  # for years
# y = []  # for occurences in that year

# model = PolynomialRegression()
# model.fit(X, y)

# plt.scatter(X, y, color = 'b')
# plt.plot(X, model.predict(X), color = 'r') # to visualize the data and verify it's good :)

# X_predict = n # the year you want to predict occurrences for
# y_predict = model.predict(X_predict) # the actual prediction yay!!

# y_predict = y_predict * [percent from month] * percent from day